## Sentiment Analysis - Amazon baby data

In [2]:
import pandas as pd
products = pd.read_csv(f"D:/Docs/amazon_baby.csv")
#products = pd.read_csv(f"D:/SYED/data/amazon_baby.csv")
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183531 entries, 0 to 183530
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   name    183213 non-null  object
 1   review  182702 non-null  object
 2   rating  183531 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 4.2+ MB


## Remove the NAN values in review column

In [3]:
products = products.fillna({'review':''})  # fill in N/A's in the review column
products['review'].isnull().values.any()

False

## Remove punctuation marks

In [4]:
import string
products['review_clean'] = products['review'].apply(lambda x: x.translate(str.maketrans('', '', 
                           string.punctuation)))
products.head(3)

,name,review,rating,review_clean
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,These flannel wipes are OK but in my opinion n...
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...


### Remove rating of 3

In [5]:
products = products[products['rating'] != 3]
products.head(3)

,name,review,rating,review_clean
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...


### More than 3 is +1 else -1

In [6]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products['review'][27401]

'I love this product.Simple but does the job great.Very easy to attach.I really have nothing bad to say about it.My baby is now protected from the sun.'

## Load the data as test and train
* `train-idx.json` for Training data
* `test-idx.json` for Test data

In [7]:
train_data_indices = pd.read_json(f"D:/repos/CourseraPlus/1_ML_Combined_Courses/Course/Week-2/train-idx.json")
train_data = pd.DataFrame(products, index = train_data_indices[0])
test_data_indices = pd.read_json(f"D:/repos/CourseraPlus/1_ML_Combined_Courses/Course/Week-2/test-idx.json")
test_data = pd.DataFrame(products, index = test_data_indices[0])
print("Train set: " + str(train_data.shape))
print("Test set: " + str(test_data.shape))

Train set: (133416, 5)
Test set: (33336, 5)


### Remove all the NaN before carrying out training

In [8]:
train_data.dropna(subset = ["name"], inplace=True)
train_data.isnull().values.sum()
test_data.dropna(subset = ["name"], inplace=True)
test_data.isnull().values.sum()
print("Train set: " + str(train_data.shape))
print("Test set: " + str(test_data.shape))

Train set: (121011, 5)
Test set: (30209, 5)


## Dictionary of word counts

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern = r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
test_matrix = vectorizer.transform(test_data['review_clean'])
print("Train matrix: " + str(train_matrix.shape))
print("Test matrix: " + str(test_matrix.shape))

Train matrix: (121011, 113128)
Test matrix: (30209, 113128)


## Logistic Regression Model

In [10]:
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression(random_state=0).fit(train_matrix, train_data['sentiment'])
zero_elem = (sentiment_model.coef_ > 0).sum()
less_elem = (sentiment_model.coef_ <= 0).sum()
total_val = zero_elem + less_elem
print("Co-efficients with value more than 0: " + str(zero_elem) + " out of " + str(total_val))

Co-efficients with value more than 0: 83399 out of 113128


C:\Users\SYED\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [45]:
sample_test_data = test_data[45:50]
sample_test_data['review_clean'][46]

'Ive tried this for almost 6 months at different points of our kid teething and shes just not that into this Its really heavy for a baby When I would hold the vibrating setting on my kid would watch for a bit and even try it but just seemed bored with it and would move on The yellow part also attracts dust and fiber out of the air like a magnet It is ALWAYS dirty Im not that particular when it comes to dirt and baby stuff but it grosses me out Its also only wire clean so its a more high maintenance toy'

In [12]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores_sample = sentiment_model.decision_function(sample_test_matrix)
print(scores_sample)

[ 3.87525176 -1.9278382   7.61510201  2.29146002  4.28449738]


### Mapping function based on scores

In [13]:
def check_score(input_array):
    result = []
    for x in input_array:
        if x >= 0:
            result.append(1)
        else:
            result.append(-1)
    return result
check_score(scores_sample)

[1, -1, 1, 1, 1]

### Check probabilities

In [14]:
prob_val = sentiment_model.predict_proba(sample_test_matrix)
print(prob_val[:,1])

[0.97967266 0.12699005 0.99950729 0.90816729 0.98640678]


## Full data sentiment analysis

In [15]:
#predictions = sentiment_model.predict(test_matrix)
final_scores_data = vectorizer.transform(test_data['review_clean'])
final_score = sentiment_model.decision_function(final_scores_data)
df = pd.DataFrame(final_score, columns = ['Score'])
print(df.shape)
print(df.columns)

(30209, 1)
Index(['Score'], dtype='object')


In [16]:
df_test = test_data
df_test.reset_index(drop = True, inplace = True)
df2 = pd.concat([df, df_test], axis = 1)
df_export = df2.sort_values(by='Score', ascending=False)
df_export.head(5)

,Score,name,review,rating,review_clean,sentiment
20952,71.340496,Joovy Scooter Single Stroller Greenie,"***I've posted an UPDATE at the end***First, l...",5.0,Ive posted an UPDATE at the endFirst let me st...,1.0
22392,57.553975,"Zooper 2011 Waltz Standard Stroller, Flax Brown",I did a TON of research before I purchased thi...,5.0,I did a TON of research before I purchased thi...,1.0
28615,53.675193,Ubbi Cloth Diaper Pail Liner,"(updated 3.22.13) After extensive research, tr...",5.0,updated 32213 After extensive research trial a...,1.0
18044,44.201557,"Infantino Wrap and Tie Baby Carrier, Black Blu...",I bought this carrier when my daughter was abo...,5.0,I bought this carrier when my daughter was abo...,1.0
9166,41.352666,"Joovy Zoom 360 Swivel Wheel Jogging Stroller, ...",The joovy zoom 360 was the perfect solution fo...,5.0,The joovy zoom 360 was the perfect solution fo...,1.0


## Export data (Top & Bot 20)

In [17]:
df_top = df_export.head(20)
#df_top.to_csv('top20.csv')
df_bottom = df_export.tail(20)
#df_bottom.to_csv('bottom20.csv')

## Calculate Accuracy - Sentiment Model

In [39]:
print("Scores dataframe shape: " + str(df.shape))
print("Test data sentiment as reference shape: " + str(test_data.shape))
accuracy_sentiment_model = sentiment_model.score(final_scores_data, test_data['sentiment'].values)
print("Sentiment model accuracy:\n" + str(accuracy_sentiment_model))

Scores dataframe shape: (30209, 1)
Test data sentiment as reference shape: (30209, 5)
Sentiment model accuracy:
0.9323711476712238


## Special Classifier

In [19]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [20]:
vectorizer_word_subset = CountVectorizer(vocabulary = significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])
print("Train matrix special: " + str(train_matrix_word_subset.shape))
print("Test matrix special: " + str(test_matrix_word_subset.shape))

Train matrix special: (121011, 20)
Test matrix special: (30209, 20)


## Simple Log-Reg model

In [21]:
simple_model = LogisticRegression(random_state=0).fit(train_matrix_word_subset, train_data['sentiment'])
zero_elem_simple = (simple_model.coef_ > 0).sum()
print(zero_elem_simple)

10


In [22]:
arr1 = simple_model.coef_
arr2 = sentiment_model.coef_
arr1 = arr1.reshape(-1, 1)
arr2 = arr2.reshape(-1, 1)
list1 = arr1.tolist()
list2 = arr2.tolist()
len(list2)

113128

In [23]:
vectorizer.get_feature_names_out()

array(['0', '000', '0001', ..., 'zzzs', 'zzzzzs', 'zzzzzzz'], dtype=object)

### Co-efficient dictionary for significant words

In [24]:
# Check the coefficients for 20 words used in model
output_dict = dict(zip(significant_words, list1))
setiment_dict = dict(zip(vectorizer.get_feature_names_out(), list2))
#print(setiment_dict)

## Compare two dictionaries

In [25]:
def compare_ndic(src, dest):
    result = []
    for skey, sval in src.items():
        for nkey, nval in dest.items():
            if skey == nkey:
                result.append([skey, sval, nval])
    return result

In [28]:
new_list = compare_ndic(output_dict, setiment_dict)
df_dict = pd.DataFrame(new_dict, columns=['Word', 'Small', 'Sentiment'])
df_dict.head(20)

,Word,Small,Sentiment
0,love,[1.362111286958226],[1.4248973032583496]
1,great,[0.9261265484135796],[1.1619452285773577]
2,easy,[1.1839658168374054],[1.2366066796719366]
3,old,[0.12316404245309119],[0.115798788918508]
4,little,[0.49928669653559854],[0.6105745963708901]
5,perfect,[1.516243780634871],[1.4768103524355587]
6,loves,[1.7448865646572689],[1.4299072376805848]
7,well,[0.47048103843759953],[0.512458487500583]
8,able,[0.18216586532103615],[0.3581624131776271]
9,car,[0.09178103094711236],[0.26452360732226937]


# Compare `train` & `test` accuracy

In [48]:
accuracy_sentiment_model = sentiment_model.score(final_scores_data, test_data['sentiment'].values)
print("Sentiment model accuracy TRAINING data:\n" + str(accuracy_sentiment_model))

accuracy_simple_model = simple_model.score(test_matrix_word_subset, test_data['sentiment'].values)
print("Simple model accuracy TRAINING data:\n" + str(accuracy_simple_model))

Sentiment model accuracy TRAINING data:
0.9323711476712238
Simple model accuracy TRAINING data:
0.8686815187526896


In [51]:
products['sentiment'].value_counts()

 1    140259
-1     26493
Name: sentiment, dtype: int64

In [53]:
print("Majority classifier accuracy should be:\n" + str(140259/(140259+26493)))

Majority classifier accuracy should be:
0.8411233448474381
